# Kaggle Competition Example

In [4]:
import os
import zipfile
from snowflake.snowpark import Session
import pandas as pd


import os
from kaggle.api.kaggle_api_extended import KaggleApi
from typing import List


In [ ]:

class InstacartDataLoader:
    def __init__(self, session: Session):
        self.session = session
        
    def unzip_data_files(self, path: str):
        """Unzip all CSV files in the directory"""
        print("Unzipping data files...")
        for file in os.listdir(path):
            if file.endswith('.zip'):
                zip_path = os.path.join(path, file)
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(path)
                print(f"Unzipped: {file}")
                
    def load_csv_to_snowflake(self, file_path: str, table_name: str, schema: str = 'INSTACART_RAW'):
        """Load CSV file to Snowflake table"""
        print(f"Loading {file_path} to {schema}.{table_name}")
        chunk_size = 100000
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            df = self.session.create_dataframe(chunk)
            df.write.save_as_table(f"{schema}.{table_name}", mode="append")
            
    def setup_incremental_loading(self):
        """Setup tables for incremental loading"""
        self.session.sql("""
            CREATE OR REPLACE TABLE INSTACART_RAW.ORDER_PRODUCTS_STAGE (
                order_id INTEGER,
                product_id INTEGER,
                add_to_cart_order INTEGER,
                reordered INTEGER,
                file_name VARCHAR,
                loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            )
        """).collect()


In [19]:


def download_instacart_data(path: str = "./data") -> List[str]:
    """
    Download Instacart Market Basket Analysis data
    
    Args:
        path: Directory to save the data
        
    Returns:
        List of downloaded file paths
    
    Raises:
        Exception: If competition rules haven't been accepted or other API errors
    """
    try:
        api = KaggleApi()
        api.authenticate()
        
        # Create data directory if it doesn't exist
        os.makedirs(path, exist_ok=True)
        
        # Check if we've already downloaded the data
        zip_path = f"{path}/instacart-market-basket-analysis.zip"
        if os.path.exists(zip_path):
            print("Data already downloaded, using existing files...")
        else:
            print("Downloading competition data...")
            try:
                api.competition_download_files(
                    'instacart-market-basket-analysis',
                    path=path
                )
            except Exception as e:
                if "rules" in str(e):
                    raise Exception(
                        "Please accept the competition rules first at "
                        "https://www.kaggle.com/competitions/instacart-market-basket-analysis"
                    ) from e
                raise
        
        # Unzip if needed
        csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
        if not csv_files:
            print("Extracting files...")
            import zipfile
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(path)
        
        # List available files
        csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
        print("\nAvailable files:")
        for file in csv_files:
            print(f"- {file}")
            
        return [os.path.join(path, f) for f in csv_files]
        
    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        raise


In [20]:

# Usage
try:
    files = download_instacart_data()
    print(f"\nSuccessfully downloaded {len(files)} files")
except Exception as e:
    print(f"Failed to download data: {str(e)}")


Extracting files...

Available files:
- sample_submission.csv

Successfully downloaded 1 files


In [24]:

class InstacartDataLoader:
    def __init__(self, session: Session, data_path: str = "./data"):
        self.session = session
        self.data_path = data_path
        
    @staticmethod
    def prep_df(df):
        """Prepare DataFrame for Snowflake loading"""
        df = df.reset_index(drop=True)
        df.index = pd.RangeIndex(start=0, stop=len(df))
        df.columns = [col.upper() for col in df.columns]
        return df
        
    def load_data(self):
        """Load all data using Snowpark DataFrames"""
        print("Loading data...")
        
        # Define all tables and their primary keys
        tables_info = {
            "aisles.csv": ("AISLES", ["AISLE_ID"]),
            "departments.csv": ("DEPARTMENTS", ["DEPARTMENT_ID"]),
            "products.csv": ("PRODUCTS", ["PRODUCT_ID"]),
            "orders.csv": ("ORDERS", ["ORDER_ID"]),
            "order_products__prior.csv": ("ORDER_PRODUCTS", ["ORDER_ID", "PRODUCT_ID"]),
            "order_products__train.csv": ("ORDER_PRODUCTS", ["ORDER_ID", "PRODUCT_ID"])
        }
        
        # Create schema
        self.session.sql("CREATE SCHEMA IF NOT EXISTS INSTACART_RAW").collect()
        
        # Load each file
        for file, (table, pk_cols) in tables_info.items():
            print(f"Processing {file}...")
            
            # Read and prep data
            df = self.prep_df(pd.read_csv(f"{self.data_path}/{file}"))
            
            # Create or append to table
            mode = "overwrite" if not file.startswith("order_products") else "append"
            self.session.create_dataframe(df) \
                .write.mode(mode) \
                .save_as_table(f"INSTACART_RAW.{table}")
            
            # Add primary key if not appending
            if mode == "overwrite":
                pk_str = ", ".join(pk_cols)
                try:
                    self.session.sql(f"""
                        ALTER TABLE INSTACART_RAW.{table} 
                        ADD PRIMARY KEY ({pk_str})
                    """).collect()
                except Exception as e:
                    print(f"Warning: Could not add primary key to {table}: {str(e)}")
    
    def validate_load(self):
        """Validate the loaded data"""
        print("\nValidating loaded data...")
        for table in ["AISLES", "DEPARTMENTS", "PRODUCTS", "ORDERS", "ORDER_PRODUCTS"]:
            count = self.session.sql(f"SELECT COUNT(*) as cnt FROM INSTACART_RAW.{table}").collect()[0]['CNT']
            print(f"{table}: {count:,} rows")


In [25]:
from snowflake_feature_store.connection import get_connection

# Specify all connection parameters directly
database = "DATASCIENCE"
schema = "INSTACART_RAW"
warehouse = "DS_WH_XS"
role = "DATA_SCIENTIST"

# Get connection with all custom parameters
conn = get_connection(
    database=database, 
    schema=schema, 
    warehouse=warehouse,
    role=role
)

# Usage
loader = InstacartDataLoader(conn.session)
loader.load_data()
loader.validate_load()


2025-02-26 19:24:58,340 - snowflake_feature_store - INFO - Using active Snowflake session
2025-02-26 19:24:58,343 - snowflake_feature_store - INFO - Initialized connection to "DATASCIENCE"."INSTACART_RAW"
2025-02-26 19:25:00,057 - snowflake_feature_store - INFO - Using role: "DATA_SCIENTIST", warehouse: "DS_WH_XS", database: DATASCIENCE, schema: INSTACART_RAW
Loading data...
Processing aisles.csv...
Processing departments.csv...
Processing products.csv...
Processing orders.csv...
Processing order_products__prior.csv...
Processing order_products__train.csv...

Validating loaded data...
AISLES: 134 rows
DEPARTMENTS: 21 rows
PRODUCTS: 49,688 rows
ORDERS: 3,421,083 rows
ORDER_PRODUCTS: 33,819,106 rows


In [17]:

from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)


In [18]:
from snowflake_feature_store.connection import get_connection

# Specify all connection parameters directly
database = "DATASCIENCE"
schema = "INSTACART_RAW"
warehouse = "DS_WH_XS"
role = "DATA_SCIENTIST"

# Get connection with all custom parameters
conn = get_connection(
    database=database, 
    schema=schema, 
    warehouse=warehouse,
    role=role
)
session = conn.session

2025-03-03 17:01:12,343 - snowflake_feature_store - INFO - Using active Snowflake session
2025-03-03 17:01:12,344 - snowflake_feature_store - INFO - Initialized connection to "DATASCIENCE"."INSTACART_RAW"
2025-03-03 17:01:13,463 - snowflake_feature_store - INFO - Using role: "DATA_SCIENTIST", warehouse: "DS_WH_XS", database: DATASCIENCE, schema: INSTACART_RAW


In [19]:
import snowflake.snowpark as snowpark
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)


In [20]:
# Initialize Feature Store
fs = FeatureStore(
    session=session, 
    database="DATASCIENCE", 
    name="INSTACART_FEATURES", 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)


In [21]:
# Register a combined entity
training_entity = Entity(name="TRAINING_INSTANCE", join_keys=["ORDER_ID", "USER_ID", "PRODUCT_ID"])
fs.register_entity(training_entity)

/Users/jdemlow/miniconda3/envs/feature-store/lib/python3.11/site-packages/snowflake/ml/feature_store/feature_store.py:197: UserWarning: Entity TRAINING_INSTANCE already exists. Skip registration.
  return f(self, *args, **kargs)


Entity(name=TRAINING_INSTANCE, join_keys=['ORDER_ID', 'USER_ID', 'PRODUCT_ID'], owner=None, desc=)

In [37]:
# Create a feature view directly from your training features table
training_features_df = session.table("DATASCIENCE.INSTACART_RAW_MARTS.INSTACART__TRAINING_FEATURES")

import snowflake.snowpark.functions as F
# Add a date column derived from order_dow
training_features_df = training_features_df.with_column(
    "synthetic_date",
    F.dateadd('day', F.col("order_dow"), F.to_date(F.lit('2023-01-01')))
)

training_features_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"USER_ID"  |"PRODUCT_ID"  |"ORDER_ID"  |"ORDER_NUMBER"  |"ORDER_DOW"  |"ORDER_HOUR_OF_DAY"  |"REORDERED"  |"USER_TOTAL_ORDERS"  |"AVG_DAYS_BETWEEN_ORDERS"  |"TYPICAL_ORDER_HO

In [38]:
# Register as a feature view
training_features_fv = FeatureView(
    name="instacart_training_features", 
    entities=[training_entity],
    feature_df=training_features_df,
    timestamp_col="synthetic_date",  # Assuming order_dow can serve as a timestamp
    refresh_freq=None,
    desc="Complete set of Instacart features for reorder prediction"
)


In [39]:

# Register the feature view
training_features_fv = fs.register_feature_view(
    feature_view=training_features_fv,
    version="1",
    # block=True,
    overwrite=True
)

In [42]:
# Generate a dataset (optional if you're using the table directly)
spine_df = session.sql("""
    SELECT 
        order_id,
        user_id,
        product_id,
        order_dow
    FROM DATASCIENCE.INSTACART_RAW_MARTS.INSTACART__TRAINING_FEATURES
    WHERE order_number > 10  -- Example filter for specific use case
""")

spine_df = spine_df.with_column(
    "synthetic_date",
    F.dateadd('day', F.col("order_dow"), F.to_date(F.lit('2023-01-01')))
)

# Drop the order_dow column
spine_df = spine_df.drop("order_dow")

spine_df.show()

------------------------------------------------------------
|"ORDER_ID"  |"USER_ID"  |"PRODUCT_ID"  |"SYNTHETIC_DATE"  |
------------------------------------------------------------
|1923543     |53002      |47310         |2023-01-07        |
|2457806     |18330      |3849          |2023-01-01        |
|1567974     |173408     |42356         |2023-01-07        |
|285366      |177221     |49488         |2023-01-07        |
|145707      |12944      |39781         |2023-01-05        |
|2183816     |187923     |3307          |2023-01-02        |
|59458       |173231     |42450         |2023-01-07        |
|2237858     |3857       |8138          |2023-01-04        |
|3377313     |190097     |13176         |2023-01-07        |
|1031250     |112968     |15227         |2023-01-03        |
------------------------------------------------------------



In [ ]:
training_dataset = fs.generate_dataset(
    name="instacart_reorder_training",
    version="1_0_1",
    spine_df=spine_df,
    features=[training_features_fv],
    spine_timestamp_col="synthetic_date",
    spine_label_cols=["reordered"]
)

Dataset(
  name='DATASCIENCE.INSTACART_FEATURES.instacart_reorder_training',
  version='1_0_1',
)

In [46]:
training_dataset.read.to_snowpark_dataframe().show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"USER_ID"  |"PRODUCT_ID"  |"SYNTHETIC_DATE"  |"ORDER_NUMBER"  |"ORDER_DOW"  |"ORDER_HOUR_OF_DAY"  |"REORDERED"  |"USER_TOTAL_ORDERS"  |"AVG_DAYS_BETWEEN_ORDE